# Data Cleaning with Python (Part 1)

In this topic, we’ll perform data cleaning tasks in Python using Pandas, focusing on:

- **Standardising data types** to ensure correct formats.
- **Standardising product names and categories** for consistency.
- **Handling missing values** to maintain data completeness.

After reading the instructions for the step, run the cell to see the results.
 


### Step 1: Import Libraries and Load Data

First, we need to import the necessary libraries and load our dataset. Pandas is the primary library we’ll use to manipulate our data.

In [ ]:
# Importing the Pandas library
import pandas as pd

# Loading the dataset
df = pd.read_csv('../inputs/datasets/raw/Customer_Sales_Transaction.xls')

# Displaying the first few rows of the dataset
df.head()


### Step 2: Standardise Data Types

Ensuring each column has the correct data type is crucial. For example, quantities should be numeric, not formatted as currency. We’ll review the data types and correct any mistakes.

- **Check Data Types:** Use `.dtypes` to view the data type of each column.
- **Convert the Quantity Column:** If `Quantity` is not already numeric, we’ll convert it to a numeric format.


In [ ]:
# Check the data types of each column
print(df.dtypes)

In Pandas, the `object` data type is a general-purpose type that usually holds strings (text data) but can technically store any Python object. When a column is labeled as `object`, it typically means:

- **Text (String) Data**: The most common usage of `object` in Pandas is for columns containing text. For example, names, product categories, and descriptions are often stored as `object` type.

- **Mixed Data**: If a column contains a mix of data types (e.g., numbers and strings together), Pandas defaults to the `object` type since it’s more flexible and can accommodate different types.

The columns that need corrections include **Transaction Date** and **Quantity**.



### Correcting Data Types

1. **Transaction Date**: Convert to `datetime` format to enable date-specific operations.
2. **Quantity**: Convert to `int` or `float`, depending on the nature of the data.

Here’s how you might apply these corrections in Python:


### a) Converting `Transaction Date` Data Type to Datetime Format

In [ ]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
print(df.dtypes)

- `pd.to_datetime(..., errors='coerce')`: Converts `Transaction Date` to `datetime` format, setting invalid dates to `NaT` (Not a Time).

In [ ]:
print("The transaction date data type is: ", df['Transaction Date'].dtype)

### b) Converting Quantity data type to `float`

In [ ]:
# Remove the pound symbol and convert the Quantity column to float
df['Quantity'] = df['Quantity'].replace('[£]', '', regex=True)
df['Quantity'] = df['Quantity'].astype(float)
print(df.dtypes)

**Explanation:**

- `.replace('[£]', '', regex=True)`: Removes the `£` symbol from each entry in the `Quantity` column. The `regex=True` argument allows for pattern matching, so any occurrence of `£` is replaced with an empty string.
- `.astype(float)`: Converts the `Quantity` column to a `float` data type after removing the currency symbol, making it ready for numerical analysis.


In [ ]:
print("The Quantity data type is: ", df['Quantity'].dtype)

In [ ]:
#print the dataset
df.head(5)

### Step 3: Standardise Text in Product Name and Product Categories

Consistency is essential when analysing categories or product names.

- **View Unique Values**: Check unique entries in `Product Name` and `Product Category`.
- **Replace Inconsistent Values**: Use `.replace()` to standardise the values.


### a) Standardise Text in Product Name
 
Inconsistent product names can interfere with grouping and analysis. For example, `Lap Top` and `Laptop` should be treated as the same product. We’ll first check for unique product names and then correct any inconsistencies.

- **View Unique Product Names:** Use `.unique()` to get a list of all unique product names.


In [ ]:
# print unique value in the column 'Product Name'
print(df['Product Name'].unique())

- **Replace Inconsistent Values:** Use `.replace()` to correct any misspelled or misformatted names.

In [ ]:
# Replace inconsistent product names
df['Product Name'] = df['Product Name'].replace({
    'Lap Top': 'Laptop'
})

- #### Check the unique values to ensure the replacements are done correctly.

In [ ]:
# Print Unique Values again
print(df['Product Name'].unique())

### b) Standardise Text in Product Category

Like product names, categories should also be consistent to prevent analysis issues. For instance, `electronic` and `Electronics` should be the same category. We’ll use similar steps as above to check and standardise `Product Category`.

- **View Unique Categories:** Use `.unique()` to see all entries in `Product Category`.

In [ ]:
print(df['Product Category'].unique())

- **Replace Inconsistent Values:** Use `.replace()` to standardize category names.

In [ ]:
# Replace 'electronic' with 'Electronics'
df['Product Category'] = df['Product Category'].replace('Electronic', 'Electronics')

In [ ]:
# Replace 'furnitures' with 'Furniture'
df['Product Category'] = df['Product Category'].replace('furnitures', 'Furniture')

- #### Check the unique values to ensure the replacements are done correctly.

In [ ]:
print(df['Product Category'].unique())

- We can see that the replacement has been done correctly, but we have noticed a missing value represented by NaN. Now, let's handle the missing values.


### Step 4: Handle Missing Values

Let's identify any missing values and handle them appropriately.

1. **Check for Missing Values:** Use `.isnull().sum()` to get a count of missing values in each column.
2. **Fill Missing Values in Product Category:** Based on context, fill in missing categories (e.g., if `Product Name` is "Tablet," set `Product Category` to "Electronics").
3. **Impute Missing Customer Ages:** Use the median age to fill in missing values in `Customer Age` as it's less affected by outliers.

In [ ]:
# Counting the number of missing values for each column
print(df.isnull().sum()) 

- We can see that the columns **Product Category** and **Customer Age** each have one missing value. Lets inspect these rows. 

### a) Handling Missing Value in Product Category Column

In [ ]:
# Print rows where 'Product Category' is null
df[df['Product Category'].isnull()]

- Here, we see that the product is "Tablet." Therefore, we can handle this missing value by setting the `Product Category` to "Electronics."

In [ ]:
import numpy as np #import the Numpy Library

# Replace NaN values in 'Product Category' with 'Electronics'
df['Product Category'] = df['Product Category'].replace(np.nan, 'Electronics')

- `np.nan` is used to represent missing values (NaN) in the replace function.
- This code replaces all `NaN` values in the Product Category column with "Electronics."

In this case, we knew there was only one missing value in `Product Category` and that the `Product Name` was "Tablet." Therefore, we used a simple replacement to fill in "Electronics" as the category:

```python
import numpy as np

# Replace NaN values in 'Product Category' with 'Electronics' for a single known missing entry
df['Product Category'] = df['Product Category'].replace(np.nan, 'Electronics')

However, if there are multiple missing values in `Product Category` for different products, each belonging to a distinct category, we need to be more specific. This approach requires us to target both the product name and the intended category to avoid incorrect replacements.

For instance, if we have missing values across several products, we can use a conditional approach to assign the correct category based on `Product Name`. Here’s how we could achieve this:

```python
# Fill missing 'Product Category' values based on specific 'Product Name' conditions

# For rows where 'Product Name' is 'Tablet' and 'Product Category' is missing, set 'Product Category' to 'Electronics'
df.loc[(df['Product Name'] == 'Tablet') & (df['Product Category'].isnull()), 'Product Category'] = 'Electronics'

# For rows where 'Product Name' is 'Chair' and 'Product Category' is missing, set 'Product Category' to 'Furniture'
df.loc[(df['Product Name'] == 'Chair') & (df['Product Category'].isnull()), 'Product Category'] = 'Furniture'

# For rows where 'Product Name' is 'Phone' and 'Product Category' is missing, set 'Product Category' to 'Electronics'
df.loc[(df['Product Name'] == 'Phone') & (df['Product Category'].isnull()), 'Product Category'] = 'Electronics'


### b) Handling Missing Value in Customer Age Column

In [ ]:
# Print rows where 'Customer Age' is null
df[df['Customer Age'].isnull()]

- For missing `'Customer Age'`, use the median to fill in these values

In [ ]:
# Finding the Median
age_median = df['Customer Age'].median()
print(age_median) # The median is 46

In [ ]:
#Replace NA values in Customer Age column with median age
df['Customer Age'] = df['Customer Age'].fillna(age_median)

In [ ]:
# Confirm no missing values remain in these key columns
print(df.isnull().sum())

## Save the Cleaned Dataset 

In [ ]:
# Export the DataFrame to a CSV file
df.to_csv('cleaned_data.csv', index=False)


In this topic, we:

- Corrected data types.
- Standardised text values in the `Product Name` and `Product Category` columns.
- Handled missing values by filling with appropriate values where possible.

By cleaning the data this way, we ensure our dataset is consistent and ready for analysis.


## What's Next?

In the next topic, Data Cleaning in Python (Part 2), we’ll focus on handling duplicates, detecting outliers, and performing data integration and feature engineering to further improve data quality.